<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >

# Hierarchical Indexing

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ``Series`` and ``DataFrame`` objects, respectively.
Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys.
While Pandas does provide ``Panel`` and ``Panel4D`` objects that natively handle three-dimensional and four-dimensional data (see [Aside: Panel Data](#Aside:-Panel-Data)), a far more common pattern in practice is to make use of *hierarchical indexing* (also known as *multi-indexing*) to incorporate multiple index *levels* within a single index.
In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.

In this section, we'll explore the direct creation of ``MultiIndex`` objects, considerations when indexing, slicing, and computing statistics across multiply indexed data, and useful routines for converting between simple and hierarchically indexed representations of your data.

We begin with the standard imports:

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

In [2]:
pd.__version__

'1.0.3'

In [3]:
np.__version__

'1.18.3'


## A Multiply Indexed Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional ``Series``.
For concreteness, we will consider a series of data where each point has a character and numerical key.

### The bad way

Suppose you would like to track data about states from two different years.
Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [4]:
ind = [('AP', 2000), ('AP', 2010),
         ('KA', 2000), ('KA', 2010),
         ('TN', 2000), ('TN', 2010),
      ('TS',2000),('TS',2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561,
              2504789,5689752]

In [5]:
pop = pd.Series(populations, index=ind)
pop

(AP, 2000)    33871648
(AP, 2010)    37253956
(KA, 2000)    18976457
(KA, 2010)    19378102
(TN, 2000)    20851820
(TN, 2010)    25145561
(TS, 2000)     2504789
(TS, 2010)     5689752
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [6]:
pop[('AP', 2000)]

33871648

In [7]:
pop[0]

33871648

In [8]:
pop[('AP', 2010):('TS', 2000)]

(AP, 2010)    37253956
(KA, 2000)    18976457
(KA, 2010)    19378102
(TN, 2000)    20851820
(TN, 2010)    25145561
(TS, 2000)     2504789
dtype: int64

In [9]:
pop[0:5]

(AP, 2000)    33871648
(AP, 2010)    37253956
(KA, 2000)    18976457
(KA, 2010)    19378102
(TN, 2000)    20851820
dtype: int64

In [ ]:
pop.index

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [10]:
pop[[i for i in pop.index if i[1] == 2010]]  # List Comp

(AP, 2010)    37253956
(KA, 2010)    19378102
(TN, 2010)    25145561
(TS, 2010)     5689752
dtype: int64

In [11]:
for i in pop.index:  
    if i[1]==2010:
        print(i,pop[i])

('AP', 2010) 37253956
('KA', 2010) 19378102
('TN', 2010) 25145561
('TS', 2010) 5689752


This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

### The Better Way: Pandas MultiIndex
Fortunately, Pandas provides a better way.
Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas ``MultiIndex`` type gives us the type of operations we wish to have.
We can create a multi-index from the tuples as follows:

In [12]:
pop.index

Index([('AP', 2000), ('AP', 2010), ('KA', 2000), ('KA', 2010), ('TN', 2000),
       ('TN', 2010), ('TS', 2000), ('TS', 2010)],
      dtype='object')

In [13]:
index = pd.MultiIndex.from_tuples(ind)
index

MultiIndex([('AP', 2000),
            ('AP', 2010),
            ('KA', 2000),
            ('KA', 2010),
            ('TN', 2000),
            ('TN', 2010),
            ('TS', 2000),
            ('TS', 2010)],
           )

Notice that the ``MultiIndex`` contains multiple *levels* of indexing–in this case, the state names and the years, as well as multiple *labels* for each data point which encode these levels.

If we re-index our series with this ``MultiIndex``, we see the hierarchical representation of the data:

In [14]:
pop = pop.reindex(index)
pop

AP  2000    33871648
    2010    37253956
KA  2000    18976457
    2010    19378102
TN  2000    20851820
    2010    25145561
TS  2000     2504789
    2010     5689752
dtype: int64

In [15]:
pop['AP']

2000    33871648
2010    37253956
dtype: int64

In [16]:
pop[0]

33871648

In [17]:
pop[2]

18976457

In [18]:
pop[0:4]

AP  2000    33871648
    2010    37253956
KA  2000    18976457
    2010    19378102
dtype: int64

In [19]:
pop[0::2]

AP  2000    33871648
KA  2000    18976457
TN  2000    20851820
TS  2000     2504789
dtype: int64

In [20]:
pop[::-1]

TS  2010     5689752
    2000     2504789
TN  2010    25145561
    2000    20851820
KA  2010    19378102
    2000    18976457
AP  2010    37253956
    2000    33871648
dtype: int64

Here the first two columns of the ``Series`` representation show the multiple index values, while the third column shows the data.
Notice that some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it.

Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

In [21]:
pop[:,2010]

AP    37253956
KA    19378102
TN    25145561
TS     5689752
dtype: int64

The result is a singly indexed array with just the keys we're interested in.
This syntax is much more convenient (and the operation is much more efficient!) than the home-spun tuple-based multi-indexing solution that we started with.
We'll now further discuss this sort of indexing operation on hieararchically indexed data.

### MultiIndex as extra dimension

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.
In fact, Pandas is built with this equivalence in mind. The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:

In [22]:
pop.ndim

1

In [23]:
pop

AP  2000    33871648
    2010    37253956
KA  2000    18976457
    2010    19378102
TN  2000    20851820
    2010    25145561
TS  2000     2504789
    2010     5689752
dtype: int64

In [24]:
pop_df = pop.unstack()
pop_df

,2000,2010
AP,33871648,37253956
KA,18976457,19378102
TN,20851820,25145561
TS,2504789,5689752


In [25]:
pop_df.ndim

2

Naturally, the ``stack()`` method provides the opposite operation:

In [26]:
Se = pop_df.stack()
Se

AP  2000    33871648
    2010    37253956
KA  2000    18976457
    2010    19378102
TN  2000    20851820
    2010    25145561
TS  2000     2504789
    2010     5689752
dtype: int64

In [27]:
Se.ndim

1

Seeing this, you might wonder why would we would bother with hierarchical indexing at all.
The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional ``Series``, we can also use it to represent data of three or more dimensions in a ``Series`` or ``DataFrame``.
Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a ``MultiIndex`` this is as easy as adding another column to the ``DataFrame``:

In [28]:
pop

AP  2000    33871648
    2010    37253956
KA  2000    18976457
    2010    19378102
TN  2000    20851820
    2010    25145561
TS  2000     2504789
    2010     5689752
dtype: int64

In [29]:
pop_df = pd.DataFrame({'Total': pop,
                       'Under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014,
                                   4789423, 5897821
                                  ]})
pop_df

Total  Under18
AP 2000  33871648  9267089
   2010  37253956  9284094
KA 2000  18976457  4687374
   2010  19378102  4318033
TN 2000  20851820  5906301
   2010  25145561  6879014
TS 2000   2504789  4789423
   2010   5689752  5897821

In [30]:
pop_df['Total']

AP  2000    33871648
    2010    37253956
KA  2000    18976457
    2010    19378102
TN  2000    20851820
    2010    25145561
TS  2000     2504789
    2010     5689752
Name: Total, dtype: int64

In addition, all the ufuncs and other functionality discussed in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) work with hierarchical indices as well.
Here we compute the fraction of people under 18 by year, given the above data:

In [31]:
f_u18 = pop_df['Under18'] / pop_df['Total']
f_u18.unstack()

,2000,2010
AP,0.273594,0.249211
KA,0.247010,0.222831
TN,0.283251,0.273568
TS,1.912106,1.036569


This allows us to easily and quickly manipulate and explore even high-dimensional data.

## Methods of MultiIndex Creation

The most straightforward way to construct a multiply indexed ``Series`` or ``DataFrame`` is to simply pass a list of two or more index arrays to the constructor. For example:

In [32]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.264271  0.184487
  2  0.650428  0.734804
b 1  0.115190  0.676071
  2  0.634368  0.906981

The work of creating the ``MultiIndex`` is done in the background.

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a ``MultiIndex`` by default:

In [33]:
data = {('Ap', 2000): 33871648,
        ('Ap', 2010): 37253956,
        ('TS', 2000): 20851820,
        ('TS', 2010): 25145561,
        ('TN', 2000): 18976457,
        ('TN', 2010): 19378102}
pd.Series(data)

Ap  2000    33871648
    2010    37253956
TS  2000    20851820
    2010    25145561
TN  2000    18976457
    2010    19378102
dtype: int64

Nevertheless, it is sometimes useful to explicitly create a ``MultiIndex``; we'll see a couple of these methods here.

### Explicit MultiIndex constructors

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the ``pd.MultiIndex``.
For example, as we did before, you can construct the ``MultiIndex`` from a simple list of arrays giving the index values within each level:

In [ ]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

You can construct it from a list of tuples giving the multiple index values of each point:

In [ ]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

You can even construct it from a Cartesian product of single indices:

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

Similarly, you can construct the ``MultiIndex`` directly using its internal encoding by passing ``levels`` (a list of lists containing available index values for each level) and ``labels`` (a list of lists that reference these labels):

In [ ]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Any of these objects can be passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``, or be passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.

### MultiIndex level names

Sometimes it is convenient to name the levels of the ``MultiIndex``.
This can be accomplished by passing the ``names`` argument to any of the above ``MultiIndex`` constructors, or by setting the ``names`` attribute of the index after the fact:

In [34]:
pop.index.names = ['state', 'year']
pop

state  year
AP     2000    33871648
       2010    37253956
KA     2000    18976457
       2010    19378102
TN     2000    20851820
       2010    25145561
TS     2000     2504789
       2010     5689752
dtype: int64

With more involved datasets, this can be a useful way to keep track of the meaning of various index values.

### MultiIndex for columns

In a ``DataFrame``, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.
Consider the following, which is a mock-up of some (somewhat realistic) medical data:

In [35]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2, 3]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Ram', 'Shan', 'Sam'], ['BMI', 'Temp','Weight']],
                                     names=['Name_pas', 'type_dis'])

In [36]:
# mock some data
data = np.round(np.random.randn(6, 9), 2)
data

array([[ 1.03,  0.63,  0.27,  1.77,  1.26,  0.33,  1.61,  1.46, -1.01],
       [-0.7 ,  0.88,  0.76, -0.54,  1.93,  2.13,  0.06,  1.77,  0.36],
       [-2.77,  1.17,  1.26,  0.01,  1.45, -1.24, -1.44, -1.79,  0.37],
       [-0.26,  1.84, -0.09,  1.57,  0.6 , -1.86,  0.56, -0.21,  1.17],
       [-0.1 , -0.77, -1.62, -0.18,  0.4 ,  1.47,  0.35, -1.07,  0.1 ],
       [ 1.19,  1.47,  1.17,  0.67,  0.1 ,  1.32, -0.51,  0.46,  0.34]])

In [37]:
data[:, ::2] *= 10
data

array([[ 1.03e+01,  6.30e-01,  2.70e+00,  1.77e+00,  1.26e+01,  3.30e-01,
         1.61e+01,  1.46e+00, -1.01e+01],
       [-7.00e+00,  8.80e-01,  7.60e+00, -5.40e-01,  1.93e+01,  2.13e+00,
         6.00e-01,  1.77e+00,  3.60e+00],
       [-2.77e+01,  1.17e+00,  1.26e+01,  1.00e-02,  1.45e+01, -1.24e+00,
        -1.44e+01, -1.79e+00,  3.70e+00],
       [-2.60e+00,  1.84e+00, -9.00e-01,  1.57e+00,  6.00e+00, -1.86e+00,
         5.60e+00, -2.10e-01,  1.17e+01],
       [-1.00e+00, -7.70e-01, -1.62e+01, -1.80e-01,  4.00e+00,  1.47e+00,
         3.50e+00, -1.07e+00,  1.00e+00],
       [ 1.19e+01,  1.47e+00,  1.17e+01,  6.70e-01,  1.00e+00,  1.32e+00,
        -5.10e+00,  4.60e-01,  3.40e+00]])

In [38]:
data += 37
data

array([[47.3 , 37.63, 39.7 , 38.77, 49.6 , 37.33, 53.1 , 38.46, 26.9 ],
       [30.  , 37.88, 44.6 , 36.46, 56.3 , 39.13, 37.6 , 38.77, 40.6 ],
       [ 9.3 , 38.17, 49.6 , 37.01, 51.5 , 35.76, 22.6 , 35.21, 40.7 ],
       [34.4 , 38.84, 36.1 , 38.57, 43.  , 35.14, 42.6 , 36.79, 48.7 ],
       [36.  , 36.23, 20.8 , 36.82, 41.  , 38.47, 40.5 , 35.93, 38.  ],
       [48.9 , 38.47, 48.7 , 37.67, 38.  , 38.32, 31.9 , 37.46, 40.4 ]])

In [43]:
# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

Name_pas     Ram                 Shan                Sam              
type_dis     BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2013 1      47.3  37.63   39.7  38.77  49.6  37.33  53.1  38.46   26.9
     2      30.0  37.88   44.6  36.46  56.3  39.13  37.6  38.77   40.6
     3       9.3  38.17   49.6  37.01  51.5  35.76  22.6  35.21   40.7
2014 1      34.4  38.84   36.1  38.57  43.0  35.14  42.6  36.79   48.7
     2      36.0  36.23   20.8  36.82  41.0  38.47  40.5  35.93   38.0
     3      48.9  38.47   48.7  37.67  38.0  38.32  31.9  37.46   40.4

In [44]:
health_data.to_csv("Health_Data_For_Multiindexing.csv")

Here we see where the multi-indexing for both rows and columns can come in *very* handy.
This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number.
With this in place we can, for example, index the top-level column by the person's name and get a full ``DataFrame`` containing just that person's information:

In [46]:
health_data['Ram']

type_dis     BMI   Temp  Weight
year visit                     
2013 1      47.3  37.63    39.7
     2      30.0  37.88    44.6
     3       9.3  38.17    49.6
2014 1      34.4  38.84    36.1
     2      36.0  36.23    20.8
     3      48.9  38.47    48.7

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

In [49]:
health_data.iloc[1] # For Rows

Name_pas  type_dis
Ram       BMI         30.00
          Temp        37.88
          Weight      44.60
Shan      BMI         36.46
          Temp        56.30
          Weight      39.13
Sam       BMI         37.60
          Temp        38.77
          Weight      40.60
Name: (2013, 2), dtype: float64

In [50]:
health_data.iloc[1:2]

Name_pas     Ram                 Shan                Sam              
type_dis     BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2013 2      30.0  37.88   44.6  36.46  56.3  39.13  37.6  38.77   40.6

## Indexing and Slicing a MultiIndex

Indexing and slicing on a ``MultiIndex`` is designed to be intuitive, and it helps if you think about the indices as added dimensions.
We'll first look at indexing multiply indexed ``Series``, and then multiply-indexed ``DataFrame``s.

### Multiply indexed Series

Consider the multiply indexed ``Series`` of state populations we saw earlier:

In [51]:
pop

state  year
AP     2000    33871648
       2010    37253956
KA     2000    18976457
       2010    19378102
TN     2000    20851820
       2010    25145561
TS     2000     2504789
       2010     5689752
dtype: int64

We can access single elements by indexing with multiple terms:

In [52]:
pop['AP', 2000]

33871648

The ``MultiIndex`` also supports *partial indexing*, or indexing just one of the levels in the index.
The result is another ``Series``, with the lower-level indices maintained:

In [53]:
pop['AP']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the ``MultiIndex`` is sorted (see discussion in [Sorted and Unsorted Indices](Sorted-and-Unsorted-Indices)):

In [55]:
pop.loc['AP':'TN']

state  year
AP     2000    33871648
       2010    37253956
KA     2000    18976457
       2010    19378102
TN     2000    20851820
       2010    25145561
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [56]:
pop[:, 2000]

state
AP    33871648
KA    18976457
TN    20851820
TS     2504789
dtype: int64


Other types of indexing and selection (discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb)) work as well; for example, selection based on Boolean masks:

In [57]:
pop[pop > 22000000]

state  year
AP     2000    33871648
       2010    37253956
TN     2010    25145561
dtype: int64

Selection based on fancy indexing also works:

In [58]:
pop[['AP', 'TN']]

state  year
AP     2000    33871648
       2010    37253956
TN     2000    20851820
       2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner.
Consider our toy medical ``DataFrame`` from before:

In [59]:
health_data

Name_pas     Ram                 Shan                Sam              
type_dis     BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2013 1      47.3  37.63   39.7  38.77  49.6  37.33  53.1  38.46   26.9
     2      30.0  37.88   44.6  36.46  56.3  39.13  37.6  38.77   40.6
     3       9.3  38.17   49.6  37.01  51.5  35.76  22.6  35.21   40.7
2014 1      34.4  38.84   36.1  38.57  43.0  35.14  42.6  36.79   48.7
     2      36.0  36.23   20.8  36.82  41.0  38.47  40.5  35.93   38.0
     3      48.9  38.47   48.7  37.67  38.0  38.32  31.9  37.46   40.4

Remember that columns are primary in a ``DataFrame``, and the syntax used for multiply indexed ``Series`` applies to the columns.
For example, we can recover Guido's heart rate data with a simple operation:

In [61]:
health_data['Ram']

type_dis     BMI   Temp  Weight
year visit                     
2013 1      47.3  37.63    39.7
     2      30.0  37.88    44.6
     3       9.3  38.17    49.6
2014 1      34.4  38.84    36.1
     2      36.0  36.23    20.8
     3      48.9  38.47    48.7

Also, as with the single-index case, we can use the ``loc``, ``iloc``, and ``ix`` indexers introduced in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb). For example:

In [62]:
health_data.iloc[:1]

Name_pas     Ram                 Shan                Sam              
type_dis     BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2013 1      47.3  37.63   39.7  38.77  49.6  37.33  53.1  38.46   26.9

In [63]:
health_data.iloc[3:-1]

Name_pas     Ram                 Shan                Sam              
type_dis     BMI   Temp Weight    BMI  Temp Weight   BMI   Temp Weight
year visit                                                            
2014 1      34.4  38.84   36.1  38.57  43.0  35.14  42.6  36.79   48.7
     2      36.0  36.23   20.8  36.82  41.0  38.47  40.5  35.93   38.0

In [64]:
health_data.iloc[:3,:-3]

Name_pas     Ram                 Shan             
type_dis     BMI   Temp Weight    BMI  Temp Weight
year visit                                        
2013 1      47.3  37.63   39.7  38.77  49.6  37.33
     2      30.0  37.88   44.6  36.46  56.3  39.13
     3       9.3  38.17   49.6  37.01  51.5  35.76

In [ ]:
health_data.iloc[-3:,:]

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices. For example:

In [66]:
health_data.loc[:, ('Ram', 'Temp')]

year  visit
2013  1        37.63
      2        37.88
      3        38.17
2014  1        38.84
      2        36.23
      3        38.47
Name: (Ram, Temp), dtype: float64

In [68]:
health_data.loc[:,('Ram')]

type_dis     BMI   Temp  Weight
year visit                     
2013 1      47.3  37.63    39.7
     2      30.0  37.88    44.6
     3       9.3  38.17    49.6
2014 1      34.4  38.84    36.1
     2      36.0  36.23    20.8
     3      48.9  38.47    48.7

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:

In [69]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-69-fb34fa30ac09>, line 1)

You could get around this by building the desired slice explicitly using Python's built-in ``slice()`` function, but a better way in this context is to use an ``IndexSlice`` object, which Pandas provides for precisely this situation.
For example:

In [70]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'Temp']]

,Name_pas,Ram,Shan,Sam
,type_dis,Temp,Temp,Temp
year,visit,,,
2013,1,37.63,49.6,38.46
2014,1,38.84,43.0,36.79


In [71]:
health_data.loc[idx[2:,2],idx[:,'BMI']]

,Name_pas,Ram,Shan,Sam
,type_dis,BMI,BMI,BMI
year,visit,,,
2013,2,30.0,36.46,37.6
2014,2,36.0,36.82,40.5


There are so many ways to interact with data in multiply indexed ``Series`` and ``DataFrame``s, and as with many tools in this book the best way to become familiar with them is to try them out!

## Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data.
There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations.
We saw a brief example of this in the ``stack()`` and ``unstack()`` methods, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we'll explore them here.

### Sorted and unsorted indices

Earlier, we briefly mentioned a caveat, but we should emphasize it more here.
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*
Let's take a look at this here.

We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:

In [72]:
index = pd.MultiIndex.from_product([['a', 'c', 'b','d'], [1, 2]])
data = pd.Series(np.random.rand(8), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.572293
      2      0.306862
c     1      0.257284
      2      0.139165
b     1      0.156876
      2      0.016845
d     1      0.436440
      2      0.773306
dtype: float64

In [73]:
data['a':'c']

UnsortedIndexError: 'Key length (1) was greater than MultiIndex lexsort depth (0)'

If we try to take a partial slice of this index, it will result in an error:

In [74]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted.
For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be in sorted (i.e., lexographical) order.
Pandas provides a number of convenience routines to perform this type of sorting; examples are the ``sort_index()`` and ``sortlevel()`` methods of the ``DataFrame``.
We'll use the simplest, ``sort_index()``, here:

In [75]:
data = data.sort_index()
data

char  int
a     1      0.572293
      2      0.306862
b     1      0.156876
      2      0.016845
c     1      0.257284
      2      0.139165
d     1      0.436440
      2      0.773306
dtype: float64

With the index sorted in this way, partial slicing will work as expected:

In [77]:
data['a':'c']

char  int
a     1      0.572293
      2      0.306862
b     1      0.156876
      2      0.016845
c     1      0.257284
      2      0.139165
dtype: float64

### Stacking and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [ ]:
pop

In [ ]:
pop.unstack()

In [ ]:
pop.unstack(level=0)

In [ ]:
pop.unstack(level=1)

The opposite of ``unstack()`` is ``stack()``, which here can be used to recover the original series:

In [ ]:
pop.unstack().stack()

In [ ]:
pop.unstack().stack(level=0)

### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the ``reset_index`` method.
Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index.
For clarity, we can optionally specify the name of the data for the column representation:

In [ ]:
pop_flat = pop.reset_index(name='population') # set the Col name
pop_flat

Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.
This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

In [ ]:
pop_flat.set_index(['state','year'])

In practice, I find this type of reindexing to be one of the more useful patterns when encountering real-world datasets.

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:

In [ ]:
health_data

Perhaps we'd like to average-out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:

In [ ]:
data_mean = health_data.mean(level='visit')
data_mean

In [ ]:
data_mean = health_data.mean(level='year')
data_mean

By further making use of the ``axis`` keyword, we can take the mean among levels on the columns as well:

In [ ]:
data_mean.mean(axis=1, level='type_dis')

Thus in two lines, we've been able to find the average heart rate and temperature measured among all subjects in all visits each year.
This syntax is actually a short cut to the ``GroupBy`` functionality, which we will discuss in [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb).
While this is a toy example, many real-world datasets have similar hierarchical structure.

<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >